In [1]:
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm, trange
from torch import nn

In [2]:
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: Xavier


In [3]:
from utils.Constants import tokenizer
from dataLoader.DataLoader import get_data_loader
from dataLoader.DataLoaderUtils import get_question_answers_for_where_value_def_length, get_question_answers_def_length

dev_data_loader = get_data_loader(data_type = 'dev', tokenizer = tokenizer, batch_size = 1)

  0%|          | 6/8421 [00:00<02:30, 55.91it/s]

We have 8421 dev data with 2716 tables.


100%|██████████| 8421/8421 [02:17<00:00, 61.16it/s]


In [4]:
from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.WhereNumberClassifier import WhereNumberClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer
from train.Trainer import train_epoch, save_model, load_model

models = dict(
    selection_trainer = SelectRankerTrainer(device, dev_data_loader),
    agg_class_trainer = AggregationClassifierTrainer(device, dev_data_loader),
    where_ranker_trainer = WhereRankerTrainer(device, dev_data_loader),
    where_cond_class_trainer = WhereConditionClassifierTrainer(device, dev_data_loader),
    where_numb_class_trainer = WhereNumberClassifierTrainer(device, dev_data_loader),
    qa_trainer = QABertTrainer(device, dev_data_loader),
)

In [5]:
#models = load_model("./checkpoint/16_Mar_2021_19_21", dev_data_loader, device)
train_epoch(models, dev_data_loader, device)
save_model(models,"./checkpoint")

  1%|▏         | 111/8421 [02:40<3:20:30,  1.45s/batch]


RuntimeError: CUDA error: device-side assert triggered